# Stock health

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Read Dataframe from output folder first


In [3]:
from pathlib import Path
import pandas as pd
from datetime import datetime

BASE_DIR = Path("/Users/andresuchitra/dev/missglam/autopo/notebook")
DATA_DIR = BASE_DIR / "output/complete"

def load_store_data(path=DATA_DIR, is_csv_only=False, is_excel_only=False):
    """
    Load and combine data from all store files, handling decimal commas correctly
    """
    all_data = []
    
    for file_path in path.glob('*.csv'):
        try:
            # Skip non-data files
            if file_path.suffix.lower() not in ['.csv', '.xlsx', '.xls']:
                continue
            # if is_csv_only, only include files with .csv extension
            if is_csv_only and file_path.suffix.lower() != '.csv':
                continue
            # if is_excel_only, only include files with .xlsx or .xls extension
            if is_excel_only and file_path.suffix.lower() not in ['.xlsx', '.xls']:
                continue

            print(f"Processing {file_path.name}...")
            
            # Read the file
            if file_path.suffix.lower() == '.csv':
                # First try to detect the delimiter
                # First detect encoding and BOM
                encoding = 'utf-8-sig'

                with open(file_path, 'rb') as f:
                    raw = f.read(4)
                    if raw.startswith(b'\xef\xbb\xbf'):
                        encoding = 'utf-8-sig'  # Handles BOM
                    else:
                        encoding = 'latin1'  # Fallback encoding

                with open(file_path, 'r', encoding=encoding) as f:
                    first_line = f.readline().strip()
                    delimiter = ';' if ';' in first_line else ','
                
                # Read with decimal comma handling
                df = pd.read_csv(
                    file_path,
                    sep=delimiter,
                    decimal=',',  # This tells pandas to use comma as decimal
                    thousands='.',  # And period as thousands separator if needed
                    encoding=encoding,
                    on_bad_lines='warn'
                )
            else:  # Excel file
                df = pd.read_excel(file_path, engine='openpyxl')
                
                # Convert string columns with decimal commas to numeric
                for col in df.select_dtypes(include=['object']).columns:
                    if df[col].astype(str).str.contains(',').any():
                        try:
                            # Try converting to float, handling decimal commas
                            df[col] = df[col].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
                        except:
                            pass  # If conversion fails, leave as is
                
            # Clean column names
            df.columns = [str(col).strip() for col in df.columns]
            
            # rename current_stock_day_cover to daily_stock_cover
            df.rename(columns={
                'Brand':'brand', 
                'current_stock_days_cover': 'daily_stock_cover', 
                'SKU': 'sku', 
                'Stok': 'stock',
                'Toko': 'store',
                'HPP': 'hpp', 
                'Harga': 'harga'}, inplace=True)
                
            all_data.append(df)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError("No valid data files could be processed")
    
    # Combine all dataframes
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined data shape: {combined_df.shape}")

    return combined_df

# Example usage:
# current_date_dir = BASE_DIR / datetime.now().strftime('%Y%m%d')
df = load_store_data(DATA_DIR,is_csv_only=True)

# set notebook to show all columns in display
pd.set_option('display.max_columns', None)

display(df)

# save to csv 'health_monitor.csv', with curren date '20251128' format
df.to_csv(f"health_monitor/{datetime.now().strftime('%Y%m%d')}.csv", index=False)

Processing 10. Miss Glam Palembang.csv...
Processing 20. Miss Glam Tanjung Pinang.csv...
Processing 1. Miss Glam Padang.csv...
Processing 6. Miss Glam Muaro Bungo.csv...
Processing 5. Miss Glam Panam.csv...
Processing 13. Miss Glam Payakumbuh.csv...
Processing 7. Miss Glam Lampung.csv...
Processing 29. Miss Glam Marpoyan.csv...
Processing 25. Miss Glam Sudirman.csv...
Processing 23. Miss Glam Halat.csv...
Processing 22. Miss Glam Pasaman Barat.csv...
Processing 16. Miss Glam Lubuk Linggau.csv...
Processing 24. Miss Glam Duri.csv...
Processing 9. Miss Glam Medan.csv...
Processing 18. Miss Glam Kedaton.csv...
Processing 8. Miss Glam Bengkulu.csv...
Processing 31. Miss Glam Mayang.csv...
Processing 15. Miss Glam Tembilahan.csv...
Processing 2. Miss Glam Pekanbaru.csv...
Processing 28. Miss Glam Aceh.csv...
Processing 27. Miss Glam P. Sidimpuan.csv...
Processing 4. Miss Glam Bukittinggi.csv...
Processing 32. Miss Glam Soeta.csv...
Processing 26. Miss Glam Dr. Mansyur.csv...
Processing 21. 

,brand,sku,Nama,store,stock,Daily Sales,Max. Daily Sales,Lead Time,Max. Lead Time,Min. Order,Sedang PO,hpp,harga,contribution_pct,contribution_ratio,Is in Padang,Orig Daily Sales,Orig Max. Daily Sales,sales_contribution,Safety stock,Reorder point,target_days_cover,qty_for_target_days_cover,current_days_stock_cover,is_open_po,initial_qty_po,emergency_po_qty,updated_regular_po_qty,final_updated_regular_po_qty,emergency_po_cost,final_updated_regular_po_cost,No,ID Supplier,Nama Supplier,ID Brand,Nama Brand,ID Store,Nama Store,Hari Order,Min. Purchase,Trading Term,Promo Factor,Delay Factor
0,ACNAWAY,10100824612,ACNAWAY Mugwort Acne Clear Bar Soap 100gr,Miss Glam Palembang,11,0.02,0.00,4,24,1,0,32445,38500,26,0.26,0,0.02,0.0,770.0,0,1,30,1,550.00,0,0,0,0,0,0,0,2796.0,10.0,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480,ACNAWAY,20,Miss Glam Palembang,2,500000,0,NaN,NaN
1,ACNAWAY,101001107647,ACNAWAY Mugwort Gel Facial Wash Mugwort + Cent...,Miss Glam Palembang,9,0.15,1.04,4,24,1,0,30900,45000,26,0.26,1,0.05,2.0,6786.0,25,26,30,5,59.68,0,0,0,0,0,0,0,2796.0,10.0,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480,ACNAWAY,20,Miss Glam Palembang,2,500000,0,NaN,NaN
2,ACNES,8992821102372,ACNES Complete White Face Wash 100gr,Miss Glam Palembang,13,0.23,1.04,3,7,3,0,26623,32500,26,0.26,1,0.20,2.0,7436.0,7,8,30,7,56.82,0,0,0,0,0,0,0,11066.0,25600.0,PT. MENSA BINASUKSES - PPN (PLB),33,ACNES,20,Miss Glam Palembang,3,500000,0,NaN,NaN
3,ACNES,8992821102365,ACNES Complete White Face Wash 50gr,Miss Glam Palembang,9,0.31,1.30,3,7,3,3,15974,19500,26,0.26,1,0.17,2.0,6084.0,9,10,30,10,28.85,1,0,0,0,0,0,0,11066.0,25600.0,PT. MENSA BINASUKSES - PPN (PLB),33,ACNES,20,Miss Glam Palembang,3,500000,0,NaN,NaN
4,ACNES,8992821100309,ACNES Creamy Wash 100gr,Miss Glam Palembang,10,0.37,1.56,3,7,3,3,25395,31000,26,0.26,1,0.31,3.0,11606.4,10,12,30,12,26.71,1,0,0,0,0,0,0,11066.0,25600.0,PT. MENSA BINASUKSES - PPN (PLB),33,ACNES,20,Miss Glam Palembang,3,500000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157580,VIO,8994942031333,VIO Acne Shield Patch Japanese Delight 12 Hydr...,Miss Glam Solok,63,0.08,1.00,1,2,3,0,14985,24000,37,0.37,0,0.08,1.0,1920.0,2,3,30,3,787.50,0,0,0,0,0,0,0,NaN,NaN,NaN,0,NaN,0,NaN,0,0,0,NaN,NaN
157581,VIO,8994942031357,VIO Acne Shield Patch Meow Meow 24 Hydrocolloi...,Miss Glam Solok,87,0.07,1.00,1,2,3,0,29970,41000,37,0.37,0,0.07,1.0,2870.0,2,3,30,3,1242.86,0,0,0,0,0,0,0,NaN,NaN,NaN,0,NaN,0,NaN,0,0,0,NaN,NaN
157582,VIO,8994942031135,VIO Acne Shield Patch Spring Blossom 20 Patches,Miss Glam Solok,42,0.02,0.37,1,2,3,0,34965,48000,37,0.37,1,0.07,1.0,888.0,1,2,30,1,2270.27,0,0,0,0,0,0,0,NaN,NaN,NaN,0,NaN,0,NaN,0,0,0,NaN,NaN
157583,VIO,8994942031296,VIO Acne Shield Patch Tuti Fruitti 12 Hydrocol...,Miss Glam Solok,87,0.05,1.00,1,2,3,0,14985,24000,37,0.37,0,0.05,1.0,1200.0,2,3,30,2,1740.00,0,0,0,0,0,0,0,NaN,NaN,NaN,0,NaN,0,NaN,0,0,0,NaN,NaN


# Master Data

In [5]:
import pandas as pd
import os

MASTER_DATA_DIR = "data/master_data"

In [ ]:
# read 'data/supplier.csv'
supplier_df = pd.read_csv('data/supplier.csv', sep=';')
display(supplier_df)

,No,ID Supplier,Nama Supplier,ID Brand,Nama Brand,ID Store,Nama Store,Hari Order,Min. Purchase,Trading Term,Promo Factor,Delay Factor
0,1,NaN,NaN,1756,BLOOD,37,PT Bersama Distriversa Indonesia,1,500000,0,NaN,NaN
1,2,NaN,NaN,875,JF THE SKIN SPECIALIST,32,Miss Glam Pasaman Barat,1,500000,0,NaN,NaN
2,3,NaN,NaN,875,JF THE SKIN SPECIALIST,34,Miss Glam Duri,1,500000,0,NaN,NaN
3,4,NaN,NaN,875,JF THE SKIN SPECIALIST,38,Miss Glam P. Sidimpuan,1,500000,0,NaN,NaN
4,5,NaN,NaN,875,JF THE SKIN SPECIALIST,39,Miss Glam Aceh,1,500000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13586,13587,3433.0,UD. WIRA JAYA SUKSES - NON PPN - SILKORO/SUMME...,357,SILKORO,40,Miss Glam Marpoyan,4,500000,0,NaN,NaN
13587,13588,3433.0,UD. WIRA JAYA SUKSES - NON PPN - SILKORO/SUMME...,357,SILKORO,41,Miss Glam Sei Penuh,4,500000,0,NaN,NaN
13588,13589,3433.0,UD. WIRA JAYA SUKSES - NON PPN - SILKORO/SUMME...,357,SILKORO,42,Miss Glam Mayang,4,500000,0,NaN,NaN
13589,13590,3421.0,VIOPAD (ONLINE),2288,VIOPAD,37,PT Bersama Distriversa Indonesia,1,500000,0,NaN,NaN


In [21]:
# create master data directory if not exists
os.makedirs(MASTER_DATA_DIR, exist_ok=True)

# generate unique store data: id (get from 'ID Store'), name
store_df = pd.DataFrame(supplier_df['ID Store'].unique(), columns=['id'])
store_df['name'] = store_df['id'].map(supplier_df.set_index('ID Store')['Nama Store'].to_dict())

store_df.sort_values(by='id', inplace=True)

store_df

# save to csv
store_df.to_csv(MASTER_DATA_DIR + '/store.csv', index=False)

# Generate master supplier data

In [29]:
# Get unique suppliers
master_supplier_df = pd.DataFrame(supplier_df['ID Supplier'].dropna().unique(), columns=['id'])
master_supplier_df['name'] = master_supplier_df['id'].map(
    supplier_df.drop_duplicates('ID Supplier').set_index('ID Supplier')['Nama Supplier'].to_dict()
)

# column 'id' as int
master_supplier_df['id'] = master_supplier_df['id'].astype(int)

display(master_supplier_df)

# save to master data directory
master_supplier_df.to_csv(MASTER_DATA_DIR + '/supplier.csv', index=False)

,id,name
0,3180,ANUGERAH NIAGA JAYA (DMI)
1,3694,ANUGERAH PHARMINDO LESTARI - PPN (BPP)
2,3343,ANUGERAH PHARMINDO LESTARI - PPN (PKU)
3,3604,APOTEK MEDIZONE - NON PPN - CASH (DC)
4,2482,ASIA WIJAYA MAKMUR - NON PPN (MDN)
...,...,...
486,2407,UD. NURIN MAKMUR - PPN (BKL)
487,2706,UD. SINAR MUTIARA BARU (PYK)
488,3433,UD. WIRA JAYA SUKSES - NON PPN - SILKORO/SUMME...
489,3421,VIOPAD (ONLINE)


# generate master brand data

In [30]:
# Get unique brand
master_brand_df = pd.DataFrame(supplier_df['ID Brand'].dropna().unique(), columns=['id'])
master_brand_df['name'] = master_brand_df['id'].map(
    supplier_df.drop_duplicates('ID Brand').set_index('ID Brand')['Nama Brand'].to_dict()
)

# column 'id' as int
master_brand_df['id'] = master_brand_df['id'].astype(int)

display(master_brand_df)

# save to master data directory
master_brand_df.to_csv(MASTER_DATA_DIR + '/brand.csv', index=False)

,id,name
0,1756,BLOOD
1,875,JF THE SKIN SPECIALIST
2,2247,JUDYDOLL
3,384,MBK
4,111,MUSTIKA RATU
...,...,...
500,716,BEAUTICA
501,1779,MONTISS
502,1734,SKIN SANE
503,149,SR 12


# Generate master products data

In [ ]:
# read product_brand_store_supplier_mappings.csv
# get unique sku in a new datafram
import os

raw_product_mapping_df = pd.read_csv(MASTER_DATA_DIR + "/product_brand_store_supplier_mappings.csv", sep=";")

HEALTH_MONITOR_DIR = Path('/Users/andresuchitra/dev/missglam/autopo/notebook/health_monitor')

unique_df = raw_product_mapping_df.drop_duplicates(subset=['sku'])

display(unique_df)

# hpp still 0, we read all csv files from health_monitor folder to read
for file_path in HEALTH_MONITOR_DIR.glob('*.csv'):
    print('\nreading...', file_path)
    df = pd.read_csv(file_path)

    # get hpp from df to unique_df by using sku as key, only check unique_df whose hpp is empty/null/NaN
    for index, row in unique_df.iterrows():
        if pd.isna(row['hpp']):
            matching_hpp = df.loc[df['sku'] == row['sku'], 'hpp'].dropna()
            if not matching_hpp.empty:
                unique_df.at[index, 'hpp'] = matching_hpp.iloc[0]

# replace any remaining NaN hpp with 0 for easier downstream handling
unique_df['hpp'] = unique_df['hpp'].fillna(0)

display("unique SKU with HPP", unique_df)

display("SKU with HPP > 0", unique_df[unique_df['hpp'] > 0])


/var/folders/8t/7219xcjd2dj829bf02_x9zy80000gn/T/ipykernel_61735/2349826740.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['hpp'] = unique_df['hpp'].fillna(0)


,brand_id,brand_name,sku,nama_product,store_id,store_name,supplier_id,supplier_name,hpp
0,68.0,EVERWHITE,28727,EVERWHITE Hicoll 120gr,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
1,68.0,EVERWHITE,30300100023,EVERWHITE Everfit Herboost,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
2,68.0,EVERWHITE,50400210010,EVERWHITE Mgxlile Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
3,68.0,EVERWHITE,50400210045,EVERWHIT Eiffel Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
4,68.0,EVERWHITE,50400210046,EVERWHITE Paris Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
...,...,...,...,...,...,...,...,...,...
703628,1760.0,COLGATE,8850006341162,COLGATE Total 12 Advance Fresh Gel Toothpaste ...,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
703629,1760.0,COLGATE,9556031203060,COLGATE Slim Soft Charcoal Sikat Gigi Isi 3,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
703630,1760.0,COLGATE,9556031312434,COLGATE Super Flexi Charcoal Soft Sikat Gigi B...,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
704118,360.0,SILCOT,8993189329333,SILCOT Maximizer Cotton Kapas Isi 40,7.0,Miss Glam Padang,3301.0,PT. RIMBUN PADI BERJAYA - PPN (PDG),0.0



reading... /Users/andresuchitra/dev/missglam/autopo/notebook/health_monitor/20251129_1.csv

reading... /Users/andresuchitra/dev/missglam/autopo/notebook/health_monitor/20251130.csv

reading... /Users/andresuchitra/dev/missglam/autopo/notebook/health_monitor/20251128_old.csv


'unique SKU with HPP'

,brand_id,brand_name,sku,nama_product,store_id,store_name,supplier_id,supplier_name,hpp
0,68.0,EVERWHITE,28727,EVERWHITE Hicoll 120gr,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
1,68.0,EVERWHITE,30300100023,EVERWHITE Everfit Herboost,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
2,68.0,EVERWHITE,50400210010,EVERWHITE Mgxlile Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
3,68.0,EVERWHITE,50400210045,EVERWHIT Eiffel Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
4,68.0,EVERWHITE,50400210046,EVERWHITE Paris Hampers Everwhite,7.0,Miss Glam Padang,54.0,PT. MIRANTI ADILLHA - PPN (PDG),0.0
...,...,...,...,...,...,...,...,...,...
703628,1760.0,COLGATE,8850006341162,COLGATE Total 12 Advance Fresh Gel Toothpaste ...,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
703629,1760.0,COLGATE,9556031203060,COLGATE Slim Soft Charcoal Sikat Gigi Isi 3,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
703630,1760.0,COLGATE,9556031312434,COLGATE Super Flexi Charcoal Soft Sikat Gigi B...,7.0,Miss Glam Padang,2879.0,PT. TIGA RAKSA (PDG),0.0
704118,360.0,SILCOT,8993189329333,SILCOT Maximizer Cotton Kapas Isi 40,7.0,Miss Glam Padang,3301.0,PT. RIMBUN PADI BERJAYA - PPN (PDG),0.0


'SKU with HPP > 0'

,brand_id,brand_name,sku,nama_product,store_id,store_name,supplier_id,supplier_name,hpp
